# Ondřej Janek: Ondra J.#0489 

## Knihovny a vytvoření dataframe

In [ ]:
!pip install h3

In [208]:
import pandas as pd
import altair as alt
import h3
import plotly.express as px
import numpy as np

In [209]:
url='https://drive.google.com/file/d/1b733K2zirixiDlZRf_McLWlBEo5pmu7L/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url, delimiter=',', decimal=',', index_col=[0])

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df["started_at"] = pd.to_datetime(df["started_at"])
df["ended_at"] = pd.to_datetime(df["ended_at"])
df['ended_at'].max() - df['started_at'].min()

In [216]:
cols = ['start_station_latitude',
        'start_station_longitude',
        'end_station_latitude',
        'end_station_longitude']
df[cols] = df[cols].astype(float)

In [217]:
df[df['start_station_id'] == 1025] = df.fillna('Corner of Dundee Street & Dundee Terrace')
df[df['end_station_id'] == 1025]   = df.fillna('Corner of Dundee Street & Dundee Terrace')

In [218]:
df[df['start_station_id'] == 1024] = df.fillna('End of Meadow Place')
df[df['end_station_id'] == 1024]   = df.fillna('End of Meadow Place')
df['start_station_name'].replace({"Meadow Place 2": "Meadow Place"}, inplace=True)
df['end_station_name'].replace({"Meadow Place 2": "Meadow Place"}, inplace=True)

In [219]:
df[df['start_station_id'] == 1092] = df.fillna('near Scottish Building Society')
df[df['end_station_id'] == 1092]   = df.fillna('near Scottish Building Society')

In [220]:
df['start_station_name'].replace({"Picady Place": "Picardy Place"}, inplace=True)
df['end_station_name'].replace({"Picady Place": "Picardy Place"}, inplace=True)

In [ ]:
df.isna().sum()

1. Dataset obsahuje 438259 řádků a 14 sloupců. Sloupce nám ukazují datum a čas výpujčky a vrácení, dobu trvání výpujčky ve vteřinách a údaje o počátečních a konečných stanic (identifikační číslo stanice, její název a popis, zeměpisnou šířku a délku)
1. Dataset začíná datem 2018-09-15 08:52:05 a končí 2021-07-01 00:20:36.
Celková délka je tedy 1019 dní 15 hodin a 28 minut.
1. Dataset obsahuje NaN hodnoty pouze v description sloupcích jak počátečních tak konečných stanic, jedná se o 3 opakující se stanice (Meadow Place 2 id = 1024, Dundee Terrace id = 1025, Dalry Road Lidl id = 1092).
* Stanici  'Dundee Terrace' chybí pouze v některých  řádcích description, proto doplním chybějící údaje.
* Stanice 'Meadow Place 2' je totožná se stanicí 'Meadow Place' a description 'End of Meadow Place', proto nahradím 'Meadow Place 2' údaji z 'Meadow Place'.
* Stanice 'Dalry Road Lidl' nemá žádné description v žádném řádku. Rozhodoval jsem se jestli sloupec description nechat prázdný nebo ho něčím nahradit. Podle zeměpisné šířky a délky jsem našel stanici na mapě, sousedí hned vedle další stanice 'Dalry Road Lid' s description 'outside Lidl', jelikož mají rozdílné ID rozhodl jsem se description stanice pojmenovat 'near Scottish Building Society'.
* V datasetu se vyskytoval jeden překlep v názvu stanice s id '2268'. Kde místo 'Picardy Place'	bylo 'Picady place'. Opraveno na správný název.

## Identifikace  aktivních a neaktivních stanic

In [222]:
# Funkce zobrazí stanice na mapě
def show_on_map(dataframe, lat, lon, name):
  fig = px.scatter_mapbox(dataframe, title = f'{len(dataframe)} stations',
                                     lat=lat,
                                     lon=lon,
                                     hover_name=name, zoom=11, height=600)
  fig.update_layout(title_x=0.5,title_y=0.95)
  fig.update_layout(mapbox_style="open-street-map")
  fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
  fig.show()

In [ ]:
# Aktivní počáteční stanice
df_start_active = df[df['started_at'] >= '2021-01-01 00:00:00']
stations = list(df_start_active['start_station_id'].unique())
df_start_active = df[df['start_station_id'].isin(stations)]
df_start_active = df_start_active.drop_duplicates(
                                    subset='start_station_id')[['start_station_name',
                                                                'start_station_description',
                                                                'start_station_latitude',
                                                                'start_station_longitude']]

show_on_map(df_start_active,
            'start_station_latitude',
            'start_station_longitude',
            'start_station_name')

In [ ]:
# Neaktivní počáteční stanice
df_start_inactive = df[~df['start_station_id'].isin(stations)]
df_start_inactive = df_start_inactive.drop_duplicates(
                                        subset='start_station_id')[['start_station_name',
                                                                    'start_station_description',
                                                                    'start_station_latitude',
                                                                    'start_station_longitude']]
                                                                    
show_on_map(df_start_inactive,
            'start_station_latitude',
            'start_station_longitude',
            'start_station_name')

In [ ]:
# Aktivní konečné stanice
df_end_active = df[df['ended_at'] >= '2021-01-01 00:00:00']
stations = list(df_end_active['end_station_id'].unique())
df_end_active = df[df['end_station_id'].isin(stations)]
df_end_active = df_end_active.drop_duplicates(
                                subset='end_station_id')[['end_station_name',
                                                          'end_station_description',
                                                          'end_station_latitude',
                                                          'end_station_longitude']]
df_end_active
show_on_map(df_end_active,
            'end_station_latitude',
            'end_station_longitude',
            'end_station_name')

In [ ]:
# Neaktivní konečné stanice
df_end_inactive = df[~df['end_station_id'].isin(stations)]
df_end_inactive = df_end_inactive.drop_duplicates(
                                    subset='end_station_id')[['end_station_name',
                                                              'end_station_description', 
                                                              'end_station_latitude', 
                                                              'end_station_longitude']]
show_on_map(df_end_inactive,
            'end_station_latitude',
            'end_station_longitude',
            'end_station_name')                                                            

1. Jako neaktivní stanice jsem definoval ty, které nebyly použity od začátku roku 2021.
2. Mapa zobrazuje aktivní a neaktivní stanice, jak počáteční tak konečné, a jejich počet.
Komentář každé buňky popisuje co mapa zobrazuje.

## Identifikace nejfrekventovanějších stanic


In [227]:
# Funkce bere jako argumenty dataframe a název sloupce. 
# Vrací DF se stanicemi a počtem výpujček nebo vrácení.
# Místo dataframe může vrátit i graf.
def count_stations(dataframe,column_name, graph = 'no'):
  dataframe['rents_num'] = dataframe.groupby(dataframe[column_name])[column_name].transform('count')
  df_frequent = dataframe[[column_name,column_name[:-3]+'_name','rents_num']]
  df_frequent = df_frequent.sort_values(by= 'rents_num', ascending= False).drop_duplicates(subset=column_name)
  if graph == 'no':
    return df_frequent
  elif graph == 'yes':
    return alt.Chart(df_frequent.head(10), title='The busiest stations').mark_bar(size=16).encode(
              x=alt.X('rents_num:Q', title='Number of rents'),
              y=alt.Y(column_name[:-3]+'_name:O', title='Stations', sort= '-x'),
              tooltip=[alt.Tooltip(column_name[:-3]+'_name:O', title="Station name"),
                       alt.Tooltip('rents_num:Q', title="Number of rents")]
          ).properties(height=300)

In [ ]:
# Graf nejfrekventovanějších počátečních stanic
count_stations(df,'start_station_id', graph='yes')   

In [ ]:
# Graf nejfrekventovanějších konečných stanic
count_stations(df,'end_station_id', graph='yes')  

In [230]:
# Získej počáteční a konečné stanice
df_start = count_stations(df,'start_station_id')
df_end = count_stations(df,'end_station_id')

In [ ]:
# Průnik 10 nejfrekventovanějších počátečních a konečných stanic
df_intersected = df_start.head(10).merge(df_end.head(10),
                                         left_on='start_station_name',
                                         right_on='end_station_name')[['start_station_name',
                                                                       'rents_num_x', 
                                                                       'rents_num_y']]

pd.DataFrame(df_intersected).rename(columns={'rents_num_x':'as_start_station',
                                             'rents_num_y':'as_end_station',
                                             'start_station_name':'station_name'})

In [ ]:
# Nejfrekventovanější stanice podle celkové sumy výpůjček a vrácení
df_sum = df_start.merge(df_end, 
                        left_on='start_station_name', 
                        right_on='end_station_name')[['start_station_name',
                                                      'rents_num_x', 
                                                      'rents_num_y']]

df_sum['rents_sum'] = df_sum['rents_num_y'] + df_sum['rents_num_x']
df_sum = df_sum.sort_values(by='rents_sum', ascending=False).rename(columns={'start_station_name':'station_name',
                                                                    'rents_num_y':'as_end_station',
                                                                    'rents_num_x':'as_start_station'}).head(10)
df_sum


In [ ]:
# Vykreslení nejfrekventovanějších stanic na mapě
stations = df_sum['station_name'].tolist()

df_frequent_map =  df.loc[df['start_station_name'].isin(stations)].drop_duplicates('start_station_name')
show_on_map(df_frequent_map,'start_station_latitude','start_station_longitude','start_station_name')

## Stanice, na kterých se kola hromadí a stanice, kde potenciálně chybí

Celkový rozdíl mezi vypůjčenými a vrácenými koly 

In [ ]:
# Získání rozdílu mezi vypůjčenými a vrácenými koly
df_diff = df[['start_station_id', 'end_station_id']].apply(pd.Series.value_counts).reset_index()
df_diff = df_diff.rename(columns={'start_station_id':'start_station_count',
                                  'end_station_id'  :'end_station_count',
                                  'index'           :'station_id'})
df_diff['difference'] = df_diff['end_station_count'].fillna(0) - df_diff['start_station_count'].fillna(0)
df_diff = df_diff.merge(df, left_on='station_id',right_on='start_station_id').drop_duplicates('station_id')
df_diff = df_diff[['start_station_name','start_station_count','end_station_count','difference']].rename(
           columns={'start_station_name':'station_name'}).sort_values(by='difference',ascending=False)

df_diff

In [ ]:
# Stanice na kterých se kola hromadí.
df_diff[df_diff['difference'] > 0][['station_name','difference']].sort_values(by='difference', ascending=False)

In [ ]:
# Stanice na kterách kola chybí.
df_diff[df_diff['difference'] < 0][['station_name','difference']].sort_values(by='difference')

Denní průměr rozdílu mezi vypůjčenými a vrácenými koly 

In [ ]:
df_day_diff = df.copy()
df_day_diff['started_at'] = df_day_diff['started_at'].dt.date
df_day_diff['ended_at']   = df_day_diff['ended_at'].dt.date
df_day_diff[df_day_diff['started_at'] != df_day_diff['ended_at']].count()

In [238]:
df_day_diff = df_day_diff[['started_at', 'start_station_id', 'end_station_id']]
df_day_diff = df_day_diff.groupby(['started_at']).agg({'start_station_id': 'value_counts',
                                                       'end_station_id'  : 'value_counts'})
df_day_diff.fillna(0, inplace=True)
df_day_diff['difference'] =  df_day_diff['end_station_id'] - df_day_diff['start_station_id']

df_day_diff = df_day_diff.reset_index().rename(
                                          columns={'started_at':'date',
                                                   'level_1'   :'station_id',
                                                   'start_station_id':'start_station_count',
                                                   'end_station_id':'end_station_count'})


In [239]:
df_day_diff = df_day_diff.reset_index().rename(
                                          columns={'started_at':'date',
                                                   'level_1'   :'station_id',
                                                   'start_station_id':'start_station_count',
                                                   'end_station_id':'end_station_count'})


In [240]:
df_day_diff = df_day_diff.groupby(['station_id']).mean().reset_index()[['station_id', 'difference']].sort_values(
              by='difference', ascending=True)

df_daily_mean = df_day_diff.merge(df,
                                  left_on='station_id',
                                  right_on='end_station_id')[['station_id',
                                                              'end_station_name',
                                                              'end_station_description',
                                                              'difference',
                                                              'end_station_latitude',
                                                              'end_station_longitude']]
df_daily_mean.drop_duplicates(subset='station_id', inplace=True)

In [ ]:
# Top 10 stanic, kde kola chybí
show_on_map(df_daily_mean.head(10),'end_station_latitude','end_station_longitude','end_station_name')

In [ ]:
df_daily_mean[df_daily_mean['difference'] < 0][['end_station_name','difference']].sort_values(by='difference')

In [ ]:
# top 10 stanic, kde kola se hromadí
show_on_map(df_daily_mean.tail(10),'end_station_latitude','end_station_longitude','end_station_name')

In [ ]:
df_daily_mean[df_daily_mean['difference'] > 0][['end_station_name','difference']].sort_values(by='difference')

1. V celém datasetu je pouze 2894 záznamů, kdy se liší datum výpujčky a vrácení.
Proto sem se rozhodl počítat denní průměr na základě "started_at_"
2. První část zobrazuje celkový rozdíl mezi vypůjčenými a vrácenými koly pro každou stanici.
3. Druhá část zobrazuje denní průměr rozdílu mezi vypůjčenými a vrácenými koly pro každou stanici.
4. Mapy ukazují TOP 10 stanic kde kola chybí a nebo se  hromadí.

## Vzdálenosti mezi jednotlivými stanicemi

In [ ]:
# Vytváření DF s názvy stanic a zeměpisné šířky a délky
df_geo = df.drop_duplicates(subset = ['end_station_id'])
df_geo['station_name'] = df_geo['end_station_name'] + ', ' + df_geo['end_station_description']
df_geo.rename(columns={'end_station_latitude':'LAT','end_station_longitude':'LON'},inplace= True)

df_geo = df_geo[['station_name','LAT','LON']]
df_geo

In [ ]:
# Vytvoření všech možných kombinací pro každou stanici
df_station_pairs = pd.DataFrame({'start':df_geo['station_name'].repeat(len(df_geo['station_name'])),
                                 'end' : [r for r in df_geo['station_name']]*len(df_geo['station_name'])})
df_station_pairs

In [ ]:
# Přiřazení zeměpisné šířky a délky pro každou stanici ve sloupcích ['start','end']
df_station_pairs = (df_station_pairs.join(df_geo.set_index('station_name').add_prefix('start_'), on='start')
                   .join(df_geo.set_index('station_name').add_prefix('end_'), on='end'))
df_station_pairs

In [248]:
# Výpočet vzdálenosti mezi stanicemi na základě LAT a LON v kilometrech
df_station_pairs['Distance'] = df_station_pairs.apply(lambda row: h3.point_dist((row['start_LAT'],
                                                                                 row['start_LON']),
                                                                                (row['end_LAT'],
                                                                                 row['end_LON']),unit='km'), axis=1)

In [ ]:
# Vytvoření matice stanice
df_distance =  df_station_pairs.pivot_table(index='start', columns='end', values='Distance')
df_distance

In [ ]:
# Seznam všech názvů stanic pro funkci níže
df_geo['station_name'].to_frame()

In [ ]:
# Funkce na pohodlnější hledání vzdáleností mezi jednotlivými stanicemi
# Funkce bere jako argument název stanice a následně zobrazí vzdálenosti všech ostatních stanic od zadané stanice
def detect_station(station_name):
  return df_distance.loc[[station_name]].transpose()
  
detect_station('Abbeyhill, Near Abbey Mount')

1. Výpočet vzdáleností mezi jednotlivými stanicemi na základě zeměpisné šířky a délky.
2. Při výpočtu vzdáleností mezi stanicemi jsem si všimnul jedné velice odlehlé stanice "Smarter Travel Station, The Street", která se nachází ve městě Liverpool místo Edinburgh

## Doba trvání jedné výpůjčky a odlehlé hodnoty.

In [ ]:
# Směrodatná odchylka v minutách
df['duration'].std()/60

In [ ]:
df['duration'].mean()/60 

In [ ]:
# Z dataframu jsem odstranil hodnoty, které byly větší než  směrodatná odchylka
# a převedl vteřiny na minuty pro lepší přehlednost
df_duration = df[df['duration'] < df['duration'].std()]
df_duration['duration'] = round(df_duration['duration']/60,0)

In [ ]:
# Průměrná doba výpůjčky 
df_duration['duration'].mean()

In [ ]:
df_duration['rents_num'] = df_duration['duration'].groupby(df_duration['duration']).transform('count')
df_duration = df_duration[['duration', 'rents_num']].drop_duplicates().sort_values(by='rents_num', ascending=False)

# Nejvíce si lidé půjčují kola v rozmezí  6 až 15 minut
df_duration.head(10)

In [ ]:
# Graf zobrazuje počet výpujček pro jednotlivé minuty
alt.Chart(df_duration, title='Time of rent').mark_bar(size=10).encode(
    x=alt.X('duration', title='Minutes'),
    y=alt.Y('rents_num', title='Number of rents'),
    tooltip=[alt.Tooltip('duration', title="Minute"),
             alt.Tooltip('rents_num', title='Number of rents')]
).properties(
    width=1200,
    height=300
) 

In [ ]:
# Top 10 nejodlehlejších hodnot
df_deviation = df.copy()
fig = px.box(df_deviation, y='duration')
fig.show()

In [ ]:
outliers = []
def detect_outliers_iqr(data):
    data = sorted(data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    IQR = q3-q1
    lwr_bound = q1-(1.5*IQR)
    upr_bound = q3+(1.5*IQR)
    for i in data: 
        if (i<lwr_bound or i>upr_bound):
            outliers.append(i)
    return outliers
sample_outliers = detect_outliers_iqr(df_deviation['duration'])
pd.DataFrame(sample_outliers)

1. Pro výpočet průměrné doby výpůjčky jsem nejdříve z datasetu odstranil hodnoty větší než směrodatná odchylka, aby nezkreslovali histogram.
2. Průměrná doba výpůjčky je 25.67 minut.
3. Nejvíce si lidé půjčují kola v rozmezí 6 až 15 minut
4. Při hledání odlehlých hodnot jsem použil vizualizaci px.box a  IQR (Inter Quartile Range). Za odhlehlé hodnoty můžeme považovat všechny doby výpůjček vyšší než 5387 vteřin (89,8 minut). Nejextrémnější hodnota je 2 363 348 vteřin což je něco málo přes 27 dní. Můžeme se tedy domnívat, že dotyčný kolo zapomněl vrátit.


## Analýza poptávky v čase

In [260]:
# Funkce zobrazí graf pro jednotlivé roky v datasetu
def show_graph(year):
  return  alt.Chart(df_month[df_month['year']==year],
          title='Number of monthly rents').mark_bar(size=18).encode(
          x=alt.X('date', title='Months',sort=None),
          y=alt.Y('Number_of_rents', title='Number of rents'),
          tooltip=[alt.Tooltip('date', title="Month"),
                  alt.Tooltip('Number_of_rents', title="Number of rents")]
        )

In [ ]:
# Dataset sem rozdělil do měsíců a spočítal pro každý měsíc počet výpůjček
data_month = df.resample('M', on='started_at').index.count()

df_month = pd.DataFrame(data_month)
df_month.reset_index(inplace=True)
df_month['year']=df_month['started_at'].dt.year
df_month['month']=df_month['started_at'].dt.month_name()
df_month['date']=df_month['month'] + ' ' + df_month['year'].astype(str)
df_month.rename(columns={'index':'Number_of_rents'},inplace=True)

alt.Chart(df_month,title='Number of monthly rents').mark_bar(size=18).encode(
    x=alt.X('date', title='Months',sort=None),
    y=alt.Y('Number_of_rents', title='Number of rents'),
    tooltip=[alt.Tooltip('date', title="Month"),
             alt.Tooltip('Number_of_rents', title="Number of rents")]
)

In [ ]:
# Roky v datasetu - 2018, 2019, 2020, 2021
show_graph(2018)

In [ ]:
show_graph(2019)

In [ ]:
show_graph(2020)

In [ ]:
show_graph(2021)

1. Z grafu vidíme, že si lidé půjčují kola nejvíc přes a jaro a léto, naopak na podzim začíná poptávka klesat.
2. Nárůst poptávky v roce 2019 mohl způsobit větší zájem lidí o bike sharing ve velkých městech, větší známost firmy než předchozí rok, nebo dobře cílená reklama.
3. Větší nárůst v roce 2020 mohl zapříčinit covid-19, kdy lidé omezovali městskou hromadnou dopravu.
4. Rok 2021 zaznamenal pokles poptávky oproti roku 2020. Opět to mohlo být spojené s  covid-19, kdy se začaly rušit různé omezení a lidé se vraceli do  městské hromadné dopravy.

## Vliv počasí na poptávku po kolech

In [266]:
url='https://drive.google.com/file/d/1Xog4IG33EZQxh6AC2Uj-ImY8NXEKrANP/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_weather = pd.read_csv(url, delimiter=',', decimal=',', index_col=[0])

In [ ]:
df_weather.isna().sum()

In [ ]:
# Vytvoření DF s denním počtem výpůjček a přídání názvů dnů
df_days = pd.DataFrame(df.resample('D', on='started_at').index.count())
df_days.reset_index(inplace=True)
df_days.rename(columns={'started_at':'date','index':'number_of_rents'},inplace=True)
df_days['name_of_day'] = df_days['date'].dt.day_name()
df_days = df_days[['date','name_of_day','number_of_rents']]
df_days

In [269]:
vis_dict = {'Excellent': 1, 'Good': 2,'Average': 3,'Poor': 4}
df_weather.replace({'vis': vis_dict}, inplace= True)

In [ ]:
# Převedené hodnot na numerické hodnoty
df_weather = (df_weather.replace(to_replace = ['°c','km/h','mm','%','mb','from','S','N','W','E'],
                                 value = '',
                                 regex= True))
cols = ['temp', 'feels', 'wind', 'gust', 'rain', 'humidity', 'cloud', 'pressure', 'vis']
df_weather[cols] = df_weather[cols].apply(pd.to_numeric)
df_weather["date"] = pd.to_datetime(df_weather["date"])
df_weather

In [271]:
# Zprůměrovaní hodnot na denní hodnoty
df_weather = df_weather.groupby('date')[cols].mean().reset_index()

In [ ]:
df_day = df_days[['date','number_of_rents']]
df_corr = pd.merge(df_weather,df_day,on='date')
df_corr

In [ ]:
# Korelace jednotlivých hodnot k výpujčkám.
df_corr.corr().loc[['number_of_rents'],cols]

1. Dataset edinburgh weather má 6336 řádků a 11 sloupců.
Sloupce ukazují čas ve 3hodinových intervalech, teplotu, pocitovou teplotu, vítr, náraz větru, déšť, vlhkost, oblačno, tlak, viditelnost a datum.
Dataset  začíná 2018-09-01 a končí 2020-10-31. Nemá žádné NaN hodnoty.
2. Pro zjištění vlivu počasí na poptávku jsem se rozhodl použít korelaci.
* Výsledky korelace nám ukazují, že pozitivní vliv na poptávku má nejvíce teplota a pocitová teplota. Což nám potvrzuje i výsledek předchozí otázky, kde je vidět, že lidé si nejvíce půjčují kola v teplých měsících.
* Naopak negativní vliv na poptávku má náraz větru, vítr a vlhkost. Dle výsledků se ale jedná o malý vliv na poptávku.
* Žádný vliv na poptávku má déšť, oblačnost a viditelnost.
* Osobně mě překvapilo, že déšť nemá na poptávku téměř žádný vliv. 
Můžeme to přisuzovat tomu, že tamní lidí jsou na déšť více zvyklý než většina populace.

## Půjčují si lidé kola více o víkendu než během pracovního týdne?

In [ ]:
# Lidé si půjčují kola více o víkendu než během pracovního týdne
days = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_days = df_days.groupby(by=["name_of_day"]).sum().reindex(days).reset_index()
df_days

In [ ]:
alt.Chart(df_days,title='The busiest days').mark_bar(size=18).encode(
    x=alt.X('name_of_day', title='Days'),
    y=alt.Y('number_of_rents', title='Number of rents'),
    tooltip=[alt.Tooltip('name_of_day', title="Day"), alt.Tooltip('number_of_rents', title="Number of rents")]
)